In [0]:
from pyspark.sql.functions import *

In [0]:
%sql 
CREATE SCHEMA tl_schema

In [0]:
%sql
CREATE TABLE tl_schema.testtable (
  id INT,
  salary INT
)
USING DELTA



In [0]:
%sql
INSERT INTO tl_schema.testtable 
VALUES (1, 100000), (2, 200000)

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
INSERT INTO tl_schema.testtable 
VALUES (3, 100000), (4, 200000)

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
INSERT INTO tl_schema.testtable 
VALUES (5, 100000), (6, 200000)

num_affected_rows,num_inserted_rows
2,2


we are keep adding small files into many partition.

In [0]:
%sql
OPTIMIZE tl_schema.testtable
ZORDER BY (id)


path,metrics
abfss://unity-catalog-storage@dbstoragetw544il4i7jwy.dfs.core.windows.net/1142509432089976/__unitystorage/catalogs/6898355c-4af8-4454-9d56-531541b7122c/tables/c7a0f13a-8a0c-41e0-b042-5b4945db08ac,"List(1, 3, List(917, 917, 917.0, 1, 917), List(865, 865, 865.0, 3, 2595), 0, List(minCubeSize(107374182400), List(0, 0), List(3, 2595), 0, List(3, 2595), 1, null), null, 0, 1, 3, 0, false, 0, 0, 1746399245350, 1746399254350, 4, 1, null, List(0, 0), 2, 2, 175, 0, null)"


**result:**
{"numFilesAdded":1,"numFilesRemoved":3,"filesAdded":{"min":917,"max":917,"avg":917,"totalFiles":1,"totalSize":917},"filesRemoved":{"min":865,"max":865,"avg":865,"totalFiles":3,"totalSize":2595},"partitionsOptimized":0,"zOrderStats":{"strategyName":"minCubeSize(107374182400)","inputCubeFiles":{"num":0,"size":0},"inputOtherFiles":{"num":3,"size":2595},"inputNumCubes":0,"mergedFiles":{"num":3,"size":2595},"numOutputCubes":1,"mergedNumCubes":null},"clusteringStats":null,"numBins":0,"numBatches":1,"totalConsideredFiles":3,"totalFilesSkipped":0,"preserveInsertionOrder":false,"numFilesSkippedToReduceWriteAmplification":0,"numBytesSkippedToReduceWriteAmplification":0,"startTimeMs":1746399245350,"endTimeMs":1746399254350,"totalClusterParallelism":4,"totalScheduledTasks":1,"autoCompactParallelismStats":null,"deletionVectorStats":{"numDeletionVectorsRemoved":0,"numDeletionVectorRowsRemoved":0},"numTableColumns":2,"numTableColumnsWithStats":2,"totalTaskExecutionTimeMs":175,"skippedArchivedFiles":0,"clusteringMetrics":null}

**OPTIMIZE = Compacts small files into larger ones to reduce file overhead (I/O).**

**ZORDER BY (id) = Reorganizes data on disk to cluster similar id values together. This improves data skipping for filter queries on id.**

💡 Z-Ordering  reorders the data within files to improve skipping and filtering during scans.


You're reducing the number of small files, which lowers the shuffle and metadata overhead.

Spark will still use multiple partitions during reading, but now can skip more irrelevant rows faster if a query filters by id.

 ZORDER BY (region, date) will cluster rows so that similar region and date combinations

Z-Ordering increases the chance that rows with similar values across multiple columns appear physically close together on disk, by reordering rows using a Z-order curve. This results in more distinct combinations being grouped, making filtering more efficient


like  multi-column filters